In [ ]:
from get_data import get_data
from blocks import init_blocks, calc_factors, get_block_stat, get_block_penalty
from students import init_students, calc_weights, get_students_penalty, update_students
from students import get_p0_students, get_students_stat, get_students_for_module, get_incomplete_students
from planner import get_try, list_results

In [ ]:
blocks = get_data('data/blocks.json')
students_tmp = get_data('data/priorities.json')
specials = get_data('data/specials.json')

N_blocks = len(blocks)
N_students = len(students_tmp)
N_tries = 1000

init_blocks(blocks)

students = init_students(students_tmp, blocks)

calc_factors(blocks)

calc_weights(students, N_blocks)

best_blocks = []

for block_nr in range(1, N_blocks+1):
    best_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
    block_penalty = get_block_penalty(best_block, block_nr, students)
    students_penalty = get_students_penalty(students, block_nr)
    penalty = block_penalty + students_penalty

    for i in range(N_tries):
        try_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
        block_penalty_try = get_block_penalty(try_block, block_nr, students)
        students_penalty_try = get_students_penalty(students, block_nr)
        penalty_try = block_penalty_try + students_penalty_try

        # if block_penalty_next < block_penalty:
        #     try_block = try_block_next
        #     block_penalty = block_penalty_next
            
        if penalty_try < penalty:
            best_block = try_block
            block_penalty = block_penalty_try
            students_penalty = students_penalty_try
            penalty = penalty_try
            
    update_students(best_block, block_nr, students)

    best_blocks.append(best_block)


In [ ]:
block_nr = 2
details = False

list_results(best_blocks, block_nr, students, details)

In [ ]:
for module in best_blocks[4]:
    print(module, get_students_for_module(best_blocks[4], module, students))

In [ ]:
get_incomplete_students(students, 5)

In [ ]:
next(s for s in students if s['ID']=='122')